In [123]:
from pandas import read_csv, DataFrame
import pandas as pd
import os
import re #Both patterns and strings to be searched can be Unicode strings as well as 8-bit strings.
import math
import cobra
import cobra.test
from __future__ import print_function
from os.path import join
from cobra.io import write_sbml_model
from cobra.flux_analysis import flux_variability_analysis
#from cobra.medium import minimal_medium
#from cobra.medium import minimal_medium
from IPython.core.interactiveshell import InteractiveShell
import itertools
import re

InteractiveShell.ast_node_interactivity = "all"

data_dir = "/Users/lizrad/Dev/iVnat"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
model=cobra.io.read_sbml_model(join(data_dir, "iVnat.xml"))
#model=cobra.io.read_legacy_sbml("/Users/lizrad/Documents/Vibrio_folder/iVnat_gapfill_glu.xml")
model

files found: 
iVnat.xml


Name,GCF_001456255_1_rast_metabolic_model
Memory address,0x01176c9630
Number of metabolites,1842
Number of reactions,1906
Objective expression,-1.0*bio1_reverse_b18f7 + 1.0*bio1
Compartments,"Cytosol_0, Extracellular_0"


## Memote biomass and consistency functions

In [2]:
from memote.support.biomass import find_blocked_biomass_precursors, find_direct_metabolites, sum_biomass_weight, find_biomass_precursors, gam_in_biomass, detect_false_positive_direct_metabolites, bundle_biomass_components, essential_precursors_not_in_biomass

In [3]:
from memote.support.consistency import check_stoichiometric_consistency

### Finding biomass precursors

In [17]:
bio_precursor=find_biomass_precursors(model, model.reactions.bio1)

print("Number of found biomass precursors:" , len(bio_precursor))
bio_precursor

Number of found biomass precursors: 76


[<Metabolite GLT_c0 at 0x1143460b8>,
 <Metabolite DEMETHYLMENAQUINONE_c0 at 0x1144960f0>,
 <Metabolite MNXM81014_c0 at 0x11459c128>,
 <Metabolite THF_c0 at 0x1144c1160>,
 <Metabolite MNXM51417_c0 at 0x114357208>,
 <Metabolite DCTP_c0 at 0x114392240>,
 <Metabolite GTP_c0 at 0x1143de240>,
 <Metabolite PUTRESCINE_c0 at 0x114389278>,
 <Metabolite ADENOSYLCOBALAMIN_c0 at 0x114714278>,
 <Metabolite CYS_c0 at 0x114346278>,
 <Metabolite MNXM5124_c0 at 0x114499278>,
 <Metabolite ILE_c0 at 0x1145842b0>,
 <Metabolite MG+2_c0 at 0x1146b92b0>,
 <Metabolite MNXM7630_c0 at 0x1147ce2b0>,
 <Metabolite CL-_c0 at 0x1144bb2e8>,
 <Metabolite FE+2_c0 at 0x1144782e8>,
 <Metabolite S-ADENOSYLMETHIONINE_c0 at 0x11436f2b0>,
 <Metabolite MNXM7294_c0 at 0x114754320>,
 <Metabolite THR_c0 at 0x1144a2320>,
 <Metabolite 5-METHYL-THF_c0 at 0x114442358>,
 <Metabolite HSO4_c0 at 0x1143de3c8>,
 <Metabolite SPERMIDINE_c0 at 0x114389438>,
 <Metabolite CPD-12822_c0 at 0x1143fd470>,
 <Metabolite NADP_c0 at 0x1143b4470>,
 <Me

### Missing essential precursors

In [100]:
missing_essential_precursors=essential_precursors_not_in_biomass(model, model.reactions.bio1)
missing_essential_precursors
len(missing_essential_precursors)

#memote looks for ids not in the annotation field, my IDs contain c0 

['MNXM94',
 'MNXM55',
 'MNXM134',
 'MNXM76',
 'MNXM61',
 'MNXM97',
 'MNXM53',
 'MNXM114',
 'MNXM42',
 'MNXM142',
 'MNXM37',
 'MNXM89557',
 'MNXM231',
 'MNXM70',
 'MNXM78',
 'MNXM199',
 'MNXM140',
 'MNXM32',
 'MNXM29',
 'MNXM147',
 'MNXM286',
 'MNXM360',
 'MNXM394',
 'MNXM344',
 'MNXM3',
 'MNXM51',
 'MNXM63',
 'MNXM121',
 'MNXM8',
 'MNXM5',
 'MNXM16',
 'MNXM33',
 'MNXM161',
 'MNXM12',
 'MNXM256',
 'MNXM119',
 'MNXM2']

37

In [102]:

list_missing_met=[]        
for met in model.metabolites:
    for mis in missing_essential_precursors:
        if mis in met.annotation.values():
           
            list_missing_met.append(met.id)
            print(met.id, mis)
            
len(list_missing_met)       
#37 vs. 52 missing metabolite--> duplication in e0/c0



ATP_c0 MNXM3
GLY_c0 MNXM29
THIAMINE-PYROPHOSPHATE_c0 MNXM256
NAD_c0 MNXM8
CPD-15815_c0 MNXM2
L-ASPARTATE_c0 MNXM42
S-ADENOSYLMETHIONINE_c0 MNXM16
MET_c0 MNXM61
SER_c0 MNXM53
CO-A_c0 MNXM12
GLT_c0 MNXM89557
CYS_c0 MNXM55
FAD_c0 MNXM33
DCTP_c0 MNXM360
LYS_c0 MNXM78
GTP_c0 MNXM51
CTP_c0 MNXM63
NADP_c0 MNXM5
DATP_c0 MNXM286
PHE_c0 MNXM97
PRO_e0 MNXM114
PRO_c0 MNXM114
PYRIDOXAL_PHOSPHATE_c0 MNXM161
HIS_c0 MNXM134
GLN_c0 MNXM37
ASN_c0 MNXM147
L-ALPHA-ALANINE_c0 MNXM32
CPD-15815_e0 MNXM2
GLT_e0 MNXM89557
ARG_c0 MNXM70
THR_c0 MNXM142
LEU_e0 MNXM140
LEU_c0 MNXM140
TTP_c0 MNXM394
ARG_e0 MNXM70
DGTP_c0 MNXM344
TYR_c0 MNXM76
FMN_c0 MNXM119
MET_e0 MNXM61
ILE_c0 MNXM231
UTP_c0 MNXM121
HIS_e0 MNXM134
ILE_e0 MNXM231
TRP_c0 MNXM94
SER_e0 MNXM53
VAL_c0 MNXM199
L-ASPARTATE_e0 MNXM42
L-ALPHA-ALANINE_e0 MNXM32
LYS_e0 MNXM78
TRP_e0 MNXM94
TYR_e0 MNXM76
VAL_e0 MNXM199


52

In [104]:
both=set()      #because of duplicate ids c0 e0  
for met in model.metabolites:
    for mis in missing_essential_precursors:
        if mis in met.annotation.values():
            both.add(mis)
            
len(both)            
print(both)            
  
#no missing essential precursors

37

{'MNXM42', 'MNXM286', 'MNXM140', 'MNXM76', 'MNXM12', 'MNXM161', 'MNXM37', 'MNXM394', 'MNXM51', 'MNXM61', 'MNXM147', 'MNXM119', 'MNXM5', 'MNXM114', 'MNXM199', 'MNXM33', 'MNXM89557', 'MNXM344', 'MNXM360', 'MNXM8', 'MNXM142', 'MNXM256', 'MNXM3', 'MNXM16', 'MNXM231', 'MNXM78', 'MNXM94', 'MNXM121', 'MNXM63', 'MNXM53', 'MNXM134', 'MNXM97', 'MNXM2', 'MNXM29', 'MNXM55', 'MNXM32', 'MNXM70'}


### Blocked biomass precursors

In [20]:
#blocked=find_blocked_biomass_precursors(model, model.reactions.bio1)

blocked_bio=find_blocked_biomass_precursors(model.reactions.bio1, model)
blocked_bio

[<Metabolite ADENOSYLCOBALAMIN_c0 at 0x114714278>,
 <Metabolite MNXM7630_c0 at 0x1147ce2b0>,
 <Metabolite ACP_c0 at 0x11433b860>]

### Sum biomass weight

In [77]:
with model:
    
    for met in model.metabolites:
        
        met.formula=re.sub("(([R])\d+)", "", met.formula)
        met.formula=re.sub("(([R]))" , "", met.formula)
        met.formula=re.sub("(MISSING)" , "", met.formula)
        print(met.formula)
    sum_bio=sum_biomass_weight(model.reactions.bio1)
    sum_bio
       
            

C7H6NO2
C7H8N5O8P2
HO7P2
H
C14H13N6O3
C10H12N5O13P3
C2H5NO2
C8H13N2O5S
C10H12N5O10P2
HO4P
C10H16N3O6S
C8H15NOS2
C16H24N4O8P2S
C12H16N4O7P2S
C12H23NO2S2
C10H11N4O8P
C5H8O14P3
C5H4N4O
C21H26N7O14P2
C3H8O2
C21H27N7O14P2
C3H6O2
H2O
C5H4NO5
C4H6NO4
CO2
C15H23N6O5S
HS
C10H17N2O3
C5H11NO2S
C10H15N2O3S
C10H13N5O3
H
C10H13N5O4
C10H13N5O4
C7H10N2O5
C5H6NO3
C2H4NO2
C31H56NO13
C6H12O6
C25H46NO8
C23H34N7O17P3S
C3H7NO3
C21H32N7O16P3S
C5H9NO4
C9H12N3O8P
C9H12N3O11P2
C4H2O4
C4H2O4
C17H25N3O17P2
C87H139N7O23P2
C9H11N2O12P2
C95H156N8O28P2
C10H8O6
C3H3O3
C7H7O4
C16H31O2
C10H12N5O7P
C37H63N7O17P3S
C5H8NO4
C3H7NO2S
C27H30N9O15P2
C4H4O5
C4H2O5
C27H33N9O15P2
C9H12N3O10P2
C9H12N3O13P3
CO2
C3H2O6P
NO2
NO2
C5H9NO3
C5H9NO3
C14H20N4O8P2S
C10H19NO2S2
C121H236O135P24
C313H548N24O255P24
C5H13N2O2
C5H13N2O2
C12H22O11
C6H12O6
C6H5O7
C6H5O7
CHNS
H4N
COS
C16H22N2O15P2
C16H22N2O15P2
C37H62N7O18P3S
C37H60N7O18P3S
C14H20N2O16P2
C21H20O10
C26H27O14
C4H10O2
C4H8O2
O2
C7H6NO3
H2O2
HO2
C14H6N2O6
C6H15N2O2
C5H16N2
C5H12O5
C5H10

C5H11NO3S
C24H35N7O18P3S
C24H33N7O18P3S
C12H22O11
C5H8NO7P
C14H27O2
C9H11FN2O6
C9H10FN2O9P
C6H9O15P3
C6H8O18P4
C27H50N9O10S2
C27H48N9O10S2
HO
C25H41O7P2
C18H32O16
C41H64N7O17P3S
C20H33O2
C30H58O13P2
C30H58O10P
C17H11O5
C27H27O5
C8H14N3O8P
C5H7O4
C18H36O9P
C67H111N4O39
C3H8O2
C3H10NO
Hg
Hg
C9H18N2O3
C27H49N2O8PS
C19H36O7P
C11H12N2O4
C10H12N2O3
C17H34O7P
C34H66O13P2
C34H66O10P
C8H11NO3
C61H101N4O34
C18H29O4
C18H27O3
C46H83N3O15P2
C40H76NO10P
C7H14O6
C63H103NO12P2
C42H76NO10P
C8H13N2O2
C42H41N4O16
C12H21O14P
N2


C8H8O
C16H19N2O5S
C16H17N2O4S
C7H6O2
C7H5O3
C7H5O6
C10H10NO5
C5H10N3
C96H170N2O38P2
C84H148N2O37P2
C6H11O7
C6H9O7
C27H42N7O18P3S
C27H40N7O18P3S
C2H8NO
C7H14N3O2
C8H16N3O2
C6H4ClO4
C6H2Cl2O4
C8H15N5O
C2H8N
C6H9N4O2
C2H2O6P
C16H28N2O11
C3H3O10P2
C9H11NO4
C9H7O5
C28H40N7O18P3S
C28H41N7O19P3S
C9H15N5O4
C28H53N2O8PS
C10H12N5O6P
C6H9N2O5
C2H3Cl3O2
C2H3Cl3O
C6H12O6
C14H25N4O11P2
C11H20NO7P
C21H29O2
C10H11N5O6P
C10H12N5O7P
C30H44O2
C31H46O2
C6H6O6
C46H70O
C2H5O3S2
C5H9O4S2
C9H18N3O2
C10H

0.9190716038475946

### Growth associated maintenance

In [107]:
#growth-associated maintenance
#True if the biomass reaction includes ATP and H2O as reactants and ADP,Pi and H as products, False otherwise.
gam=gam_in_biomass(model, model.reactions.bio1)
gam
model.reactions.bio1.reaction
#What does it check(id?)?

model.metabolites.MNXM2214_c0
model.metabolites.MNXM5258_c0
model.metabolites.MNXM90619_c0



False

'0.00309646685192537 10-FORMYL-THF_c0 + 0.00309646685192537 5-METHYL-THF_c0 + 0.00309646685192537 ACP_c0 + 0.00309646685192537 ADENOSYLCOBALAMIN_c0 + 0.246696822701341 ARG_c0 + 0.200830806928348 ASN_c0 + 40.1101757365074 ATP_c0 + 0.00309646685192537 CA+2_c0 + 0.00309646685192537 CL-_c0 + 0.00309646685192537 CO+2_c0 + 0.00309646685192537 CO-A_c0 + 0.0106480421341882 CPD-12818_c0 + 0.0106480421341882 CPD-12822_c0 + 35.5403092430435 CPD-15815_c0 + 0.00309646685192537 CPD-9728_c0 + 0.0250105977108944 CPD0-2271_c0 + 0.0841036156544863 CTP_c0 + 0.00309646685192537 CU+2_c0 + 0.0761464922056484 CYS_c0 + 0.0175915886179248 DATP_c0 + 0.0144513778992424 DCTP_c0 + 0.00309646685192537 DEMETHYLMENAQUINONE_c0 + 0.0144513778992424 DGTP_c0 + 0.00309646685192537 FAD_c0 + 0.00309646685192537 FE+2_c0 + 0.00309646685192537 FE+3_c0 + 0.219088153012743 GLN_c0 + 0.219088153012743 GLT_c0 + 0.00309646685192537 GLUTATHIONE_c0 + 0.509869786991038 GLY_c0 + 0.135406821203723 GTP_c0 + 0.0792636000737159 HIS_c0 + 0.0

Metabolite identifier,MNXM2214_c0
Name,apo_ACP
Memory address,0x01162e5240
Formula,HO
Compartment,c0
In 3 reaction(s),"MNXR95403_c0, bio1, MNXR95411_c0"


Metabolite identifier,MNXM5258_c0
Name,Peptidoglycan_polymer_n_1_subunits
Memory address,0x0116487a90
Formula,C40H63N8O21
Compartment,c0
In 2 reaction(s),"MNXR136199_c0, bio1"


Metabolite identifier,MNXM90619_c0
Name,Cobinamide
Memory address,0x01163e0588
Formula,C48H73CoN11O8
Compartment,c0
In 2 reaction(s),"MNXR134792_c0, bio1"


### False positive direct metabolites

In [111]:
false_postive=detect_false_positive_direct_metabolites(model.metabolites, model.reactions.bio1, "c0", "e0" )

TypeError: detect_false_positive_direct_metabolites() missing 2 required positional arguments: 'reaction_fluxes' and 'metabolite_fluxes'

### Bundle biomass components

In [23]:
bundle=bundle_biomass_components(model, model.reactions.bio1)
bundle

[<Reaction bio1 at 0x115af2748>]

### Find direct metabolites

In [110]:
with model: 
    for met in model.metabolites:
        if "(" in met.formula:
            print(met.formula, met.id)
            
  
    model.metabolites.MNXM8231_c0.formula="C21H33N5O9P2"
   
    model.metabolites.get_by_id("DNA-6-O-Methyl-Guanines_c0").formula="C10H16N2NO2O6S"
    model.metabolites.MNXM8231_c0.formula
    model.metabolites.get_by_id("DNA-6-O-Methyl-Guanines_c0").formula
    direct_met=find_direct_metabolites(model, model.reactions.bio1, tolerance=1e-06)
    direct_met
    len(direct_met)
    for met in model.metabolites:
        
        met.formula=re.sub("(([R])\d+)", "", met.formula)
        met.formula=re.sub("(([R]))" , "", met.formula)
        met.formula=re.sub("(MISSING)" , "", met.formula)
        
    sum_bio=sum_biomass_weight(model.reactions.bio1)
    sum_bio
    
    
    #false_postive=detect_false_positive_direct_metabolites(model.metabolites, model.reactions.bio1, "c0", "e0" )

    

'C21H33N5O9P2'

'C10H16N2NO2O6S'

[<Metabolite MN+2_c0 at 0x1164c1080>,
 <Metabolite CU+2_c0 at 0x116217128>,
 <Metabolite MNXM81014_c0 at 0x1162ca128>,
 <Metabolite ZN+2_c0 at 0x11629d1d0>,
 <Metabolite MNXM6297_c0 at 0x11620d240>,
 <Metabolite ACP_c0 at 0x116118278>,
 <Metabolite K+_c0 at 0x11646f390>,
 <Metabolite MNXM78334_c0 at 0x116338518>,
 <Metabolite CA+2_c0 at 0x1164c16a0>,
 <Metabolite MNXM7305_c0 at 0x11648e9b0>,
 <Metabolite MNXM51417_c0 at 0x116118ba8>,
 <Metabolite MNXM5124_c0 at 0x11620dc50>,
 <Metabolite MG+2_c0 at 0x116381c88>,
 <Metabolite MNXM7630_c0 at 0x116487c88>,
 <Metabolite MNXM7294_c0 at 0x116437cf8>,
 <Metabolite MNXM6175_c0 at 0x11620de80>,
 <Metabolite MNXM5129_c0 at 0x116205f98>]

17

0.9190716038475946

TypeError: detect_false_positive_direct_metabolites() missing 4 required positional arguments: 'cytosol', 'extra', 'reaction_fluxes', and 'metabolite_fluxes'

### Stoichiometric consistency

In [98]:
stoich=check_stoichiometric_consistency(model)
stoich

True

### Memote repair

from memote import test_model

In [ ]:
_, result = test_model(model, results=True)

In [ ]:
result

### Create a copy model 

In [ ]:
#test
print('original objective: ', model.objective.expression)
with model as model1:
    model1.objective = '3PGAREARR-RXN_c0'
    print('print objective in first context:', model1.objective.expression)
    with model as model2:
        model2.objective = '1.1.1.127-RXN_c0'
        print('print objective in second context:', model2.objective.expression)
    print('objective after exiting second context:',
          model2.objective.expression)
print('back to original objective:', model.objective.expression)



### Reset the exchange fluxes to zero

Change the exchange fluxes to zero. Then add the ones that are necessary for minimal media. (What if there is no media that we want? How does the media work ? Where does it show? 
Create a dictionary where there is a readable format for the medium. Is there an annotation field for medium? 

In [ ]:
#rxn = model.reactions.get_by_id

#rxn.bounds(0,0)
for reac in model.reactions:
    print(reac.id)

In [ ]:
#model.reactions.EX_cpd00009_e0.bounds = (0,1000)

    
with model:
    
    for reac in model.reactions:
        if reac.id.startswith("EX") or  ["MNXR137716_c0", 'MNXR137717_c0', 'MNXR137715_c0']:
            reac.bounds=(0,0)
    model.medium 
    model.reactions.EX_cpd00012_e0.bounds = (-1000,1000) #ppi
    model.reactions.EX_cpd00009_e0.bounds = (-1000,1000) #pi
    model.reactions.EX_cpd00209_e0.bounds = (-1000,1000) #NO3
    model.reactions.EX_cpd00075_e0.bounds = (-1000,1000) #NO2
    model.reactions.EX_cpd00013_e0.bounds = (-1000,1000) #NH4
    model.reactions.EX_cpd00048_e0.bounds = (-1000,1000) #SO4
    model.reactions.EX_cpd00027_e0.bounds = (-1000,1000) #D-glucose
    model.reactions.EX_cpd00971_e0.bounds = (-1000,1000) #Na+
    model.reactions.EX_cpd00076_e0.bounds = (-1000,1000) #Sucrose
    model.reactions.EX_cpd00080_e0.bounds = (-1000,1000) #D-Glycerol 1-phosphate
    model.reactions.EX_cpd00001_e0.bounds = (-1000,1000) #H2O
    model.reactions.EX_cpd00011_e0.bounds = (-1000,1000) #CO2
    model.reactions.EX_cpd00067_e0.bounds = (-1000,1000) #H+
    model.reactions.EX_cpd00063_e0.bounds = (-1000,1000) #Ca2+
    model.reactions.EX_cpd00099_e0.bounds = (-1000,1000) #Cl-
    model.reactions.EX_cpd00205_e0.bounds = (-1000,1000) #K+
    model.reactions.EX_cpd00254_e0.bounds = (-1000,1000) #Mg2+
    model.reactions.EX_cpd00030_e0.bounds = (-1000,1000) #Mn2+
    model.reactions.EX_cpd00007_e0.bounds = (-1000,1000) #O2
    model.reactions.MNXR137716_c0.bounds = (-1000,1000) #
    model.reactions.MNXR137715_c0.bounds = (-1000,1000) #
    model.reactions.MNXR137717_c0.bounds = (-1000,1000) #
    model.reactions.EX_cpd00058_e0.bounds = (-1000,1000) #Cu2+
    model.reactions.EX_cpd10515_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd10516_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00034_e0.bounds = (-1000,1000) #Fe2+
    
    model.reactions.EX_cpd00035_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00039_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00041_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd11580_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00023_e0.bounds = (-1000,1000) #Fe2
    model.slim_optimize()

    model.medium
    #model.optimize()
    
    model.reactions.MNXR137715_c0

In [ ]:
with model:
    
    for reac in model.reactions:
        if reac.id.startswith("EX") or  ["MNXR137716_c0", 'MNXR137717_c0', 'MNXR137715_c0']:
            reac.bounds=(0,0)
    model.medium 
    model.reactions.EX_cpd00012_e0.bounds = (-1000,0) #ppi
    model.reactions.EX_cpd00009_e0.bounds = (-1000,0) #pi
    model.reactions.EX_cpd00209_e0.bounds = (-1000,0) #NO3
    model.reactions.EX_cpd00075_e0.bounds = (-1000,0) #NO2
    model.reactions.EX_cpd00013_e0.bounds = (-1000,0) #NH4
    model.reactions.EX_cpd00048_e0.bounds = (-1000,0) #SO4
    model.reactions.EX_cpd00027_e0.bounds = (-1000,0) #D-glucose
    model.reactions.EX_cpd00971_e0.bounds = (-1000,0) #Na+
    model.reactions.EX_cpd00076_e0.bounds = (-1000,0) #Sucrose
    model.reactions.EX_cpd00080_e0.bounds = (-1000,0) #D-Glycerol 1-phosphate
    model.reactions.EX_cpd00001_e0.bounds = (-1000,0) #H2O
    model.reactions.EX_cpd00011_e0.bounds = (-1000,0) #CO2
    model.reactions.EX_cpd00067_e0.bounds = (-1000,0) #H+
    model.reactions.EX_cpd00063_e0.bounds = (-1000,0) #Ca2+
    model.reactions.EX_cpd00099_e0.bounds = (-1000,0) #Cl-
    model.reactions.EX_cpd00205_e0.bounds = (-1000,0) #K+
    model.reactions.EX_cpd00254_e0.bounds = (-1000,0) #Mg2+
    model.reactions.EX_cpd00030_e0.bounds = (-1000,0) #Mn2+
    model.reactions.EX_cpd00007_e0.bounds = (-1000,0) #O2
    model.reactions.MNXR137716_c0.bounds = (-1000,0) #
    model.reactions.MNXR137715_c0.bounds = (-1000,0) #
    model.reactions.MNXR137717_c0.bounds = (-1000,0) #
    model.reactions.EX_cpd00058_e0.bounds = (-1000,0) #Cu2+
    model.reactions.EX_cpd10515_e0.bounds = (-1000,0) #Fe2+
    model.reactions.EX_cpd10516_e0.bounds = (-1000,0) #Fe2+
    model.reactions.EX_cpd00034_e0.bounds = (-1000,0) #Fe2+
    model.medium
    model.optimize()
    model.slim_optimize()
    model.reactions.EX_cpd00012_e0

In [ ]:
with model:
    
    for reac in model.reactions:
        if reac.id.startswith("EX") or   ["MNXR137716_c0", 'MNXR137717_c0', 'MNXR137715_c0']:
            reac.bounds=(0,0)
    model.medium
    model.reactions.EX_cpd00012_e0.bounds = (0,1000) #ppi
    model.reactions.EX_cpd00009_e0.bounds = (0,1000) #pi
    model.reactions.EX_cpd00209_e0.bounds = (0,1000) #NO3
    model.reactions.EX_cpd00075_e0.bounds = (0,1000) #NO2
    model.reactions.EX_cpd00013_e0.bounds = (0,1000) #NH4
    model.reactions.EX_cpd00048_e0.bounds = (0,1000) #SO4
    model.reactions.EX_cpd00027_e0.bounds = (0,1000) #D-glucose
    model.reactions.EX_cpd00971_e0.bounds = (0,1000) #Na+
    model.reactions.EX_cpd00076_e0.bounds = (0,1000) #Sucrose
    model.reactions.EX_cpd00080_e0.bounds = (0,1000) #D-Glycerol 1-phosphate
    model.reactions.EX_cpd00001_e0.bounds = (0,1000) #H2O
    model.reactions.EX_cpd00011_e0.bounds = (0,1000) #CO2
    model.reactions.EX_cpd00067_e0.bounds = (0,1000) #H+
    model.reactions.EX_cpd00063_e0.bounds = (0,1000) #Ca2+
    model.reactions.EX_cpd00099_e0.bounds = (0,1000) #Cl-
    model.reactions.EX_cpd00205_e0.bounds = (0,1000) #K+
    model.reactions.EX_cpd00254_e0.bounds = (0,1000) #Mg2+
    model.reactions.EX_cpd00030_e0.bounds = (0,1000) #Mn2+
    model.reactions.EX_cpd00007_e0.bounds = (0,1000) #O2
    model.reactions.MNXR137716_c0.bounds = (0,1000) #
    model.reactions.MNXR137715_c0.bounds = (0,1000) #
    model.reactions.MNXR137717_c0.bounds = (0,1000) #
    model.reactions.EX_cpd00058_e0.bounds = (0,1000) #Cu2+
    model.reactions.EX_cpd10515_e0.bounds = (0,1000) #Fe2+
    model.reactions.EX_cpd10516_e0.bounds = (0,1000) #Fe2+
    model.reactions.EX_cpd00034_e0.bounds = (0,1000) #Fe2+
    model.medium
    model.optimize()
    model.slim_optimize()
    model.reactions.EX_cpd00012_e0
    

In [ ]:
model.reactions.MNXR137716_c0

In [ ]:
with model:
    for med in model.medium.keys():
        medium = model.medium
     
        medium[med] = 0.0
        model.medium = medium
        
    medium = model.medium
    medium['EX_cpd00012_e0']=1000.0 #diphosphate
    medium['EX_cpd00009_e0']= 1000 #pi .
    print(model.reactions.EX_cpd00009_e0.bounds)
    medium['EX_cpd00209_e0']=1000.0 #NO3. (CPD-15028)
    medium['EX_cpd00075_e0']=1000.0 #NO2 
    medium['EX_cpd00013_e0']=1000.0 #NH4
    
    medium['EX_cpd00048_e0']=1000.0 #SO4
    medium['EX_cpd00027_e0']=1000.0 #D-Glucose
    medium['EX_cpd00971_e0']=1000.0 #Na+ .
    medium['EX_cpd00007_e0']=1000.0 #O2
    medium['EX_cpd00076_e0']=1000.0 #Sucrose
    medium['EX_cpd00080_e0']=1000.0 #D-glycerol
    medium['EX_cpd00001_e0']=1000.0 #water
    medium['EX_cpd00011_e0']=1000.0 #CO2
    medium['EX_cpd00067_e0']=1000.0 #H
    medium['MNXR137715_c0']=1000.0
    medium['MNXR137716_c0']=1000.0
    medium['MNXR137717_c0']=1000.0
    
    medium['EX_cpd00063_e0']=1000.0 #Ca
    medium['EX_cpd00099_e0']=1000.0 #Cl
    medium['EX_cpd00205_e0']=1000.0 #K
    medium['EX_cpd00254_e0']=1000.0 #Mg
    medium['EX_cpd00030_e0']=1000.0 #Mn
    medium['EX_cpd00058_e0']=1000.0 #Cu
    medium['EX_cpd10515_e0']=1000.0 #Fe2
    medium['EX_cpd10516_e0']=1000.0 #Fe3
    medium['EX_cpd00034_e0']=1000.0 #Zn
    medium['EX_cpd00305_e0']=1000.0 #Zn
    
    medium['EX_cpd00023_e0']=1000.0 #glutamate
    medium['EX_cpd00035_e0']=1000.0 #alanin
    medium['EX_cpd00039_e0']=1000.0 #Zn
    medium['EX_cpd00041_e0']=1000.0 #Zn
    medium['EX_cpd11580_e0']=1000.0 #Zn
    

    #flux_variability_analysis(model, model.reactions)
    model.medium = medium
    model.medium
    model.slim_optimize()
    
    model.summary()
   
    #solution=model.optimize().objective_value
    
    
#model.medium
model.slim_optimize()
model.summary()




In [ ]:
flux_variability_analysis(model, model.reactions)

### Existing media and their description to dict

In [ ]:
model.exchanges
model.boundary


In [ ]:
chem_xref = read_csv("/Users/lizrad/Documents/Vibrio_folder/chem_xref.tsv" , sep='\t' , low_memory=False)
chem_xref
#499091 530559

df_seed = chem_xref.loc[499088:530556,'XREF':'Description']
pivoted_seed = df_seed.pivot_table(index='XREF',columns='XREF_ID',values='Description',aggfunc = lambda x: x)

#pivoted_seedd=pivoted_seed.drop(['metanetx'])
pivoted_seed
chem_xref_MNX_seed_dict = pivoted_seed.to_dict('index')
chem_xref_MNX_seed_dict2 = chem_xref_MNX_seed_dict['seed']
print(chem_xref_MNX_seed_dict2)


In [ ]:
#MNXR137716_c0','MNXR137717_c0','MNXR137715_c0' dont have seed IDs
medium_dict={}
for med in model.medium:
    if med.startswith("MNXR"):
        continue
    pre, seed, compartment=med.split("_")
    print(seed)
    for k in chem_xref_MNX_seed_dict2.keys():
        if seed in k: 
            print (chem_xref_MNX_seed_dict2[k])
            medium_dict[pre+"_"+seed+"_"+compartment]=chem_xref_MNX_seed_dict2[k]
            
print(medium_dict)

In [ ]:
#check every combinations of mediums :(  
#for L in range(0, len(model.medium)+1):
#    for subset in itertools.combinations(model.medium, L):
#        print(subset)

From KBase download medium in tsv. Try LB(Lysogeny Broth) and minimal nms(nitrate minimal salts medium). With Edit Media Application in KBase one can modify the existing medium) 

Result: No growth (neither).
Check for blocked precursors: both blocked with 76 metabolites. 


In [89]:
media_nms = read_csv("/Users/lizrad/Documents/Vibrio_folder/LB.tsv" , sep='\t' , low_memory=False)
#media_nms = read_csv("/Users/lizrad/Documents/Vibrio_folder/minimal_nms.tsv" , sep='\t' , low_memory=False)
media_nms

#with model:
comp_list=media_nms["compounds"].tolist()
comp_list
with model:
    for reac in model.reactions:
        reac.bounds=(0,0)
        #reac
    for reac in model.reactions:
        if reac.id.startswith("EX"):
            pre, seed, compart =reac.id.split("_")
            if seed in comp_list: 
                seed_id= pre + "_"+ seed + "_"+compart
                seed_id
                reac.bounds=(-1000,1000)
               
    len(model.medium)
    model.optimize()
    model.summary()
    
    blocked_precursors=find_blocked_biomass_precursors(model.reactions.bio1, model)
    blocked_precursors
    len(blocked_precursors)
#model.reactions.get_by_id['EX_cpd00067_e0'].bounds
    
    

,compounds,name,formula,minFlux,maxFlux,concentration
0,cpd00001,H2O,H2O,-100,100,0.001
1,cpd00007,O2,O2,-100,100,0.001
2,cpd00009,Phosphate,HO4P,-100,100,0.001
3,cpd00018,AMP,C10H13N5O7P,-100,100,0.001
4,cpd00023,L-Glutamate,C5H8NO4,-100,100,0.001
5,cpd00027,D-Glucose,C6H12O6,-100,100,0.001
6,cpd00028,Heme,C34H30FeN4O4,-100,100,0.001
7,cpd00030,Mn2+,Mn,-100,100,0.001
8,cpd00033,Glycine,C2H5NO2,-100,100,0.001
9,cpd00034,Zn2+,Zn,-100,100,0.001


['cpd00001',
 'cpd00007',
 'cpd00009',
 'cpd00018',
 'cpd00023',
 'cpd00027',
 'cpd00028',
 'cpd00030',
 'cpd00033',
 'cpd00034',
 'cpd00035',
 'cpd00039',
 'cpd00041',
 'cpd00046',
 'cpd00048',
 'cpd00051',
 'cpd00054',
 'cpd00058',
 'cpd00060',
 'cpd00063',
 'cpd00065',
 'cpd00066',
 'cpd00067',
 'cpd00069',
 'cpd00084',
 'cpd00091',
 'cpd00092',
 'cpd00099',
 'cpd00107',
 'cpd00119',
 'cpd00126',
 'cpd00129',
 'cpd00149',
 'cpd00156',
 'cpd00161',
 'cpd00182',
 'cpd00184',
 'cpd00205',
 'cpd00215',
 'cpd00218',
 'cpd00219',
 'cpd00220',
 'cpd00226',
 'cpd00239',
 'cpd00246',
 'cpd00249',
 'cpd00254',
 'cpd00311',
 'cpd00322',
 'cpd00381',
 'cpd00383',
 'cpd00393',
 'cpd00438',
 'cpd00531',
 'cpd00541',
 'cpd00644',
 'cpd00654',
 'cpd00793',
 'cpd00971',
 'cpd01012',
 'cpd01048',
 'cpd03424',
 'cpd10515',
 'cpd10516',
 'cpd11595']

'EX_cpd00067_e0'

'EX_cpd00182_e0'

'EX_cpd00129_e0'

...

43

,fluxes,reduced_costs
H2PTEROATESYNTH-RXN_c0,0.0,-4.295657e-02
GLUTATHIONE-SYN-RXN_c0,0.0,-4.295657e-02
MNXR101695_c0,0.0,-0.000000e+00
HYPOXANPRIBOSYLTRAN-RXN_c0,0.0,-4.295657e-02
RXN-8641_c0,0.0,-1.142647e-18
...,...,...
EX_cpd00011_e0,0.0,1.073914e-02
EX_cpd11416_c0,0.0,0.000000e+00
EX_cpd00099_e0,0.0,0.000000e+00
EX_cpd00063_e0,0.0,0.000000e+00


IN FLUXES    OUT FLUXES    OBJECTIVES
-----------  ------------  ------------
                           bio1  0


[<Metabolite GLT_c0 at 0x1143460b8>,
 <Metabolite DEMETHYLMENAQUINONE_c0 at 0x1144960f0>,
 <Metabolite MNXM81014_c0 at 0x11459c128>,
 <Metabolite THF_c0 at 0x1144c1160>,
 <Metabolite MNXM51417_c0 at 0x114357208>,
 <Metabolite DCTP_c0 at 0x114392240>,
 <Metabolite GTP_c0 at 0x1143de240>,
 <Metabolite PUTRESCINE_c0 at 0x114389278>,
 <Metabolite ADENOSYLCOBALAMIN_c0 at 0x114714278>,
 <Metabolite CYS_c0 at 0x114346278>,
 <Metabolite MNXM5124_c0 at 0x114499278>,
 <Metabolite ILE_c0 at 0x1145842b0>,
 <Metabolite MG+2_c0 at 0x1146b92b0>,
 <Metabolite MNXM7630_c0 at 0x1147ce2b0>,
 <Metabolite CL-_c0 at 0x1144bb2e8>,
 <Metabolite FE+2_c0 at 0x1144782e8>,
 <Metabolite S-ADENOSYLMETHIONINE_c0 at 0x11436f2b0>,
 <Metabolite MNXM7294_c0 at 0x114754320>,
 <Metabolite THR_c0 at 0x1144a2320>,
 <Metabolite 5-METHYL-THF_c0 at 0x114442358>,
 <Metabolite HSO4_c0 at 0x1143de3c8>,
 <Metabolite SPERMIDINE_c0 at 0x114389438>,
 <Metabolite CPD-12822_c0 at 0x1143fd470>,
 <Metabolite NADP_c0 at 0x1143b4470>,
 <Me

76

In [125]:
from memote.support.helpers import find_biomass_reaction
#with model:
model.reactions.bio1.annotation['SBO'] = 'SBO:0000629'
model.reactions.bio1.annotation
    #model.reactions.bio1.id="biomass"

biomass_reac=find_biomass_reaction(model)
biomass_reac

write_sbml_model(model, "/Users/lizrad/Dev/iVnat/iVnat.xml")
    

{'SBO': 'SBO:0000629'}

[<Reaction bio1 at 0x119e88cf8>]